# Proyecto Ciencia de Datos 2 – Ventas (UY)

**Enriquecimiento con datos públicos (INE/CKAN + SNIG)**  
**Fecha:** 2025-10-23

Este notebook descarga datos públicos de Uruguay y enriquece el dataset de ventas con:
- `nivel_desarrollo_regional` (combinando población y actividad económica proxy).
- `intensidad_mercado` = ventas registradas / población del departamento.

Luego realiza EDA con visualizaciones (mapas y correlaciones) y exporta un CSV enriquecido.



## Abstracto (motivación y audiencia)
Este análisis busca **explicar diferencias de ventas** entre departamentos de Uruguay, incorporando contexto regional
(obtención de **población** y una **proxy de actividad económica** por departamento) a partir de fuentes públicas
(INE/CKAN y servicios ArcGIS del SNIG).  
**Audiencia:** perfiles ejecutivos y analíticos de negocio que necesitan entender **dónde** y **por qué** se venden más/menos productos,
para orientar estrategias comerciales (mix de productos, pricing, promociones y expansión).



## Preguntas / Hipótesis
1. ¿Las ventas son **proporcionales a la población** de cada departamento?
2. ¿Los departamentos con **mayor actividad económica** (proxy: mayor cantidad de empresas registradas) muestran **más ventas**?
3. ¿Qué departamentos presentan **mayor intensidad de mercado** (ventas per cápita)?
4. ¿Cómo varía un índice simple de **desarrollo regional** (población + empresas) entre departamentos y su relación con ventas?


In [ ]:

# @title Setup (instalación de librerías) – ejecutar una sola vez en Colab/Jupyter
# Si usas Google Colab, descomenta las líneas de instalación (pip). En Jupyter local, instala una vez por terminal.
# !pip install pandas numpy requests geopandas pyproj shapely mapclassify matplotlib plotly openpyxl

import pandas as pd
import numpy as np
import requests
import io
import json
import matplotlib.pyplot as plt

# geopandas y mapclassify se usan para mapas coropléticos
try:
    import geopandas as gpd
    import mapclassify as mc
except Exception as e:
    print("Si estás en Colab, ejecuta el pip install de geopandas/mapclassify en la celda anterior.")
    raise e



## Fuentes de datos públicas (APIs/descargas)
- **Dataset de ventas (del proyecto en GitHub)**: CSV con registros de ventas en Uruguay.  
  `https://raw.githubusercontent.com/Gonrampsn4/Proyecto-Ciencia-de-Datos2-Ventas_Ramallo/main/dataset_ventas_3000.csv`

- **Población por departamento** (CSV; *Datos básicos de Juntas Departamentales* – Catálogo Nacional de Datos Abiertos):  
  `https://catalogodatos.gub.uy/dataset/7e7c97c8-a7cc-4f1f-9c85-a2c25ae28141/resource/5e1cf37b-201e-43b7-a5ba-66ee0e64e4d0/download/datosbasicosjds.csv`

  > Este CSV incluye, entre otras columnas, **Población** y el **Departamento**. (Catálogo CKAN Uruguay).

- **Proxy de actividad económica por departamento**: **conteo de empresas** por departamento (Excel público).  
  `https://catalogodatos.gub.uy/dataset/575ccb87-ae74-4dcd-ba4b-cf050bd8e08a/resource/e8e6f2e4-357e-4027-b91a-407b5d5501f7/download/empresasdei_20230330.xlsx`

  > El archivo contiene filas de empresas con su departamento. Agruparemos para obtener **empresas por departamento**.

- **Geometría de departamentos (GeoJSON)** – Servicio ArcGIS **SNIG** (consulta directa tipo API):  
  `https://web.snig.gub.uy/arcgisserver/rest/services/Uruguay/SNIG_Catastro/MapServer/2/query?where=1%3D1&outFields=Nombre&outSR=4326&f=geojson`

  > Devolverá los polígonos departamentales en GeoJSON (campo `Nombre`).


In [ ]:

# @title Descarga de datos
ventas_url = "https://raw.githubusercontent.com/Gonrampsn4/Proyecto-Ciencia-de-Datos2-Ventas_Ramallo/main/dataset_ventas_3000.csv"
poblacion_url = "https://catalogodatos.gub.uy/dataset/7e7c97c8-a7cc-4f1f-9c85-a2c25ae28141/resource/5e1cf37b-201e-43b7-a5ba-66ee0e64e4d0/download/datosbasicosjds.csv"
empresas_xlsx_url = "https://catalogodatos.gub.uy/dataset/575ccb87-ae74-4dcd-ba4b-cf050bd8e08a/resource/e8e6f2e4-357e-4027-b91a-407b5d5501f7/download/empresasdei_20230330.xlsx"
departamentos_geojson_url = "https://web.snig.gub.uy/arcgisserver/rest/services/Uruguay/SNIG_Catastro/MapServer/2/query?where=1%3D1&outFields=Nombre&outSR=4326&f=geojson"

# Ventas
ventas = pd.read_csv(ventas_url)
print("Ventas:", ventas.shape)
display(ventas.head())

# Población
poblacion = pd.read_csv(poblacion_url)
print("Población (raw):", poblacion.shape)
display(poblacion.head())

# Empresas (proxy actividad económica)
resp_emp = requests.get(empresas_xlsx_url)
empresas = pd.read_excel(io.BytesIO(resp_emp.content))
print("Empresas (raw):", empresas.shape)
display(empresas.head())

# GeoJSON departamentos (ArcGIS SNIG)
geojson = requests.get(departamentos_geojson_url).json()
gdf_deptos = gpd.GeoDataFrame.from_features(geojson["features"], crs="EPSG:4326")
gdf_deptos.rename(columns={"Nombre":"Departamento"}, inplace=True)
print("GDF deptos:", gdf_deptos.shape)
gdf_deptos.head()


In [ ]:

# @title Limpieza y estandarización de nombres de departamento

def norm_depto(s):
    if pd.isna(s):
        return s
    s = str(s).strip().upper()
    # normalizaciones comunes
    repl = {
        "SAN JOS\u00c9": "SAN JOSÉ",
        "TREINTA Y TRES": "TREINTA Y TRES",
        "MONTEVIDEO": "MONTEVIDEO",
        "CANELONES": "CANELONES",
        "MALDONADO": "MALDONADO",
        "COLONIA": "COLONIA",
        "ROCHA": "ROCHA",
        "LAVALLEJA": "LAVALLEJA",
        "FLORIDA": "FLORIDA",
        "DURAZNO": "DURAZNO",
        "SORIANO": "SORIANO",
        "RIO NEGRO": "RÍO NEGRO",
        "R\u00cdO NEGRO": "RÍO NEGRO",
        "PAYSANDU": "PAYSANDÚ",
        "PAYSAND\u00da": "PAYSANDÚ",
        "SALTO": "SALTO",
        "ARTIGAS": "ARTIGAS",
        "RIVERA": "RIVERA",
        "TACUAREMB\u00d3": "TACUAREMBÓ",
        "TACUAREMBO": "TACUAREMBÓ",
        "CERRO LARGO": "CERRO LARGO",
        "FLORES": "FLORES",
        "SAN JOSE": "SAN JOSÉ"
    }
    return repl.get(s, s)

# Inferir la columna de departamento en ventas
dept_cols = [c for c in ventas.columns if c.lower() in ["departamento","depto","region","provincia","estado"]]
if not dept_cols:
    raise ValueError("No encuentro columna de departamento en el dataset de ventas. "
                     "Renombra o crea una columna 'Departamento' con el nombre del departamento uruguayo.")
ventas["Departamento"] = ventas[dept_cols[0]].apply(norm_depto)

# Normalizar población
pop_cols = [c for c in poblacion.columns if "depar" in c.lower() or c.lower()=="departamento"]
if not pop_cols:
    raise ValueError("No encuentro columna de 'Departamento' en el CSV de población.")
poblacion["Departamento"] = poblacion[pop_cols[0]].apply(norm_depto)

# Intentar identificar la columna de población
cand_pop = [c for c in poblacion.columns if "poblac" in c.lower() or "habit" in c.lower()]
if not cand_pop:
    print("No se detectó una columna obvia de población; imprime columnas para inspección:")
    print(poblacion.columns.tolist())
    raise ValueError("Indica manualmente la columna de población en el CSV de 'datosbasicosjds'.")
poblacion.rename(columns={cand_pop[0]:"Poblacion"}, inplace=True)

# Normalizar empresas (proxy actividad) – buscar columna departamento
emp_dept_cols = [c for c in empresas.columns if "depar" in c.lower() or c.lower()=="departamento"]
if not emp_dept_cols:
    print("Columnas 'empresas':", empresas.columns.tolist())
    raise ValueError("No encuentro columna de 'Departamento' en el Excel de empresas.")
empresas["Departamento"] = empresas[emp_dept_cols[0]].apply(norm_depto)

# Agregar: conteo de empresas por departamento
empresas_por_depto = (empresas
                      .groupby("Departamento", as_index=False)
                      .size()
                      .rename(columns={"size":"Empresas"}))

# Merge poblacion + empresas
contexto_depto = (poblacion[["Departamento","Poblacion"]]
                  .merge(empresas_por_depto, on="Departamento", how="left"))
contexto_depto["Empresas"] = contexto_depto["Empresas"].fillna(0).astype(int)

contexto_depto.head()


In [ ]:

# @title Agregación de ventas por departamento + nuevas columnas

# Detectar columnas de importe y cantidad
val_cols = [c for c in ventas.columns if any(k in c.lower() for k in ["monto","importe","total","venta","revenue","price","amount"])]
qty_cols = [c for c in ventas.columns if any(k in c.lower() for k in ["cantidad","qty","units","unidades"])]

if not val_cols:
    raise ValueError("No encuentro columna de monto/importe de venta. Renombra alguna columna a 'monto' o similar.")
monto_col = val_cols[0]
print("Usando columna de valor de venta:", monto_col)

ventas_depto = (ventas
                .groupby("Departamento", as_index=False)
                .agg(ventas_total=(monto_col,"sum"),
                     operaciones=("Departamento","count"),
                     ticket_promedio=(monto_col,"mean")))

# Unir contexto
panel = ventas_depto.merge(contexto_depto, on="Departamento", how="left")

# Intensidad de mercado: ventas per cápita
panel["intensidad_mercado"] = panel["ventas_total"] / panel["Poblacion"]

# Índice simple de desarrollo regional (z-score de población y empresas)
from scipy.stats import zscore
panel["z_pop"] = zscore(panel["Poblacion"].fillna(panel["Poblacion"].median()))
panel["z_emp"] = zscore(panel["Empresas"].fillna(panel["Empresas"].median()))
panel["nivel_desarrollo_regional"] = (panel["z_pop"] + panel["z_emp"]) / 2.0

panel.sort_values("ventas_total", ascending=False).head()


In [ ]:

# @title EDA rápido (correlaciones y dispersión)
num_cols = ["ventas_total","operaciones","ticket_promedio","Poblacion","Empresas","intensidad_mercado","nivel_desarrollo_regional"]
corr = panel[num_cols].corr(numeric_only=True)

plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest')
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
plt.yticks(range(len(num_cols)), num_cols)
plt.colorbar()
plt.title("Matriz de correlaciones")
plt.tight_layout()
plt.show()

# Dispersión ventas vs empresas (proxy actividad)
plt.figure(figsize=(6,5))
plt.scatter(panel["Empresas"], panel["ventas_total"])
plt.xlabel("Empresas (proxy actividad)")
plt.ylabel("Ventas totales")
plt.title("Ventas vs. Empresas por departamento")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:

# @title Mapa coroplético: intensidad de mercado (ventas per cápita)

# Normalizar nombres en gdf
gdf_deptos["Departamento"] = gdf_deptos["Departamento"].apply(lambda s: str(s).upper())
# Unificar tildes básicas
gdf_deptos["Departamento"] = gdf_deptos["Departamento"].str.replace("RIO NEGRO","RÍO NEGRO", regex=False)
gdf_deptos["Departamento"] = gdf_deptos["Departamento"].str.replace("PAYSANDU","PAYSANDÚ", regex=False)
gdf_deptos["Departamento"] = gdf_deptos["Departamento"].str.replace("TACUAREMBO","TACUAREMBÓ", regex=False)
gdf_deptos["Departamento"] = gdf_deptos["Departamento"].str.replace("SAN JOSE","SAN JOSÉ", regex=False)

gdf_panel = gdf_deptos.merge(panel, on="Departamento", how="left")

ax = gdf_panel.plot(column="intensidad_mercado",
                    legend=True, figsize=(7,7),
                    scheme="Quantiles", k=5, linewidth=0.5, edgecolor="black")
ax.set_title("Intensidad de mercado (ventas per cápita) por departamento")
ax.axis("off")
plt.show()


In [ ]:

# @title Exportar CSV enriquecido
output_csv = "ventas_enriquecidas_uy.csv"
panel.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Archivo exportado: {output_csv}")



## Insights (resumen preliminar)
- *[Completar tras ejecutar]* ¿Qué departamentos muestran **mayor intensidad de mercado** (ventas per cápita)?  
- *[Completar]* ¿Existe correlación positiva entre **Empresas** y **Ventas**? ¿Qué tan fuerte es?  
- *[Completar]* ¿Cómo cambia el ranking si usamos el **nivel de desarrollo regional**?  
- *[Completar]* Recomendaciones comerciales (productos, promociones, expansión / cobertura).
